In [ ]:
import pandas as pd
import random
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/IPD dataset_ff.csv")

In [ ]:
df.head(10)

,PUNCTUATION,DIFFICULTY LEVEL,SENTENCE
0,!,easy,It's a beautiful day outside!
1,",",easy,"The meeting is scheduled for Monday, January 1..."
2,?,easy,"She wondered, Will it rain today?"
3,.,easy,The quick brown fox jumped over the lazy dog.
4,!,easy,"Wow, what an incredible day!"
5,",",easy,"She said, I can't make it to the party."
6,?,easy,What is the capital of France?
7,",",easy,"She ran up the stairs, burst into the room, an..."
8,!,easy,"Alas,he is dead!"
9,?,easy,"Hey,could you lend me a hand?"


In [ ]:
df['DIFFICULTY LEVEL']=df['DIFFICULTY LEVEL'].replace({'easy':1,'medium':2,'difficult':3})

In [ ]:
df

,PUNCTUATION,DIFFICULTY LEVEL,SENTENCE
0,!,1.0,It's a beautiful day outside!
1,",",1.0,"The meeting is scheduled for Monday, January 1..."
2,?,1.0,"She wondered, Will it rain today?"
3,.,1.0,The quick brown fox jumped over the lazy dog.
4,!,1.0,"Wow, what an incredible day!"
5,",",1.0,"She said, I can't make it to the party."
6,?,1.0,What is the capital of France?
7,",",1.0,"She ran up the stairs, burst into the room, an..."
8,!,1.0,"Alas,he is dead!"
9,?,1.0,"Hey,could you lend me a hand?"


In [ ]:
import random

class BayesianKnowledgeTracing:
    def __init__(self, initial_knowledge=0.5, learning_rate=0.1, guess_rate=0.2, slip_rate=0.1):
        self.knowledge = initial_knowledge
        self.learning_rate = learning_rate
        self.guess_rate = guess_rate
        self.slip_rate = slip_rate

    def update_knowledge(self, correct):
        if correct:
            p_know_given_correct = self.knowledge * (1 - self.slip_rate) / ((self.knowledge * (1 - self.slip_rate)) + ((1 - self.knowledge) * self.guess_rate))
            self.knowledge = p_know_given_correct
        else:
            p_know_given_incorrect = self.knowledge * self.slip_rate / ((self.knowledge * self.slip_rate) + ((1 - self.knowledge) * (1 - self.guess_rate)))
            self.knowledge = p_know_given_incorrect

        self.knowledge += (1 - self.knowledge) * self.learning_rate

    def next_level(self):
        if self.knowledge > 0.75:
            return 3.0
        elif self.knowledge > 0.5:
            return 2.0
        else:
            return 1.0

def calculate_score_and_select_next_sentence(level, num_punctuation, time_taken, total_attempts, total_time, correct, df, prev_indices=[], bkt_model=None):
    if bkt_model is None:
        bkt_model = BayesianKnowledgeTracing()

    bkt_model.update_knowledge(correct)

    base_scores = {'easy': 10, 'medium': 15, 'difficult': 20}
    weight_per_punctuation = {'easy': 1.0, 'medium': 2.0, 'difficult': 3.0}

    if level not in base_scores:
        raise ValueError("Invalid level provided")

    base_score = base_scores[level]
    wt_per_punctuation = weight_per_punctuation[level]

    time_bonus = 10 * (1 - time_taken / total_time)
    score = (base_score * wt_per_punctuation * num_punctuation) - total_attempts + time_bonus
    ideal_score = (base_score * wt_per_punctuation * num_punctuation) - num_punctuation + 10
    percent = score * 100 / ideal_score

    bkt_next_level = bkt_model.next_level()
    custom_algo_next_level = 3.0 if percent >= 80 else 2.0 if percent >= 50 else 1.0
    final_next_level = min(bkt_next_level, custom_algo_next_level)
    print(bkt_next_level)
    print(custom_algo_next_level)
    print(final_next_level)

    next_level_sentences = df[df['DIFFICULTY LEVEL'] == final_next_level]

    if not next_level_sentences.empty:
        available_indices = [idx for idx in next_level_sentences.index if idx not in prev_indices]

        if available_indices:
            random_index = random.choice(available_indices)
            random_sentence = next_level_sentences.loc[random_index, 'SENTENCE']
            prev_indices.append(random_index)
            return score, percent, final_next_level, random_sentence, prev_indices
        else:
            return score, percent, final_next_level, "All available sentences at the next level have been used.", prev_indices
    else:
        return score, percent, final_next_level, "No sentences found at the next level.", prev_indices

bkt_model = BayesianKnowledgeTracing()
level = 'easy'
num_punctuation = 5
time_taken = 30
total_attempts = 3
total_time = 60
correct = True

score, percent, next_level, next_sentence, prev_indices = calculate_score_and_select_next_sentence(level, num_punctuation, time_taken, total_attempts, total_time, correct, df, prev_indices=[], bkt_model=bkt_model)
print(f"Score: {score}")
print(f"Percent: {percent}%")
print(f"Next Level: {next_level}")
print(f"Next Sentence: {next_sentence}")


3.0
3.0
3.0
Score: 52.0
Percent: 94.54545454545455%
Next Level: 3.0
Next Sentence: We are trying our best, but Is it possible that we'll finish the project on time?


In [ ]:
#pfa irt-response theory bkt pybkt model ...https://files.eric.ed.gov/fulltext/ED615610.pdf